In [ ]:
import pandas as pd
from collections import defaultdict
import json

def get_category_path_string(categories):
    """카테고리 리스트를 문자열로 변환합니다."""
    return ' > '.join(categories) if categories else 'Unknown'

def find_frequently_bought_together(orders_paths, item_profiles, target_item_id, number_filter, top_n=10):
    """주어진 상품과 함께 많이 주문된 상품을 찾고, 특정 숫자로 끝나는 이름 필터링 및 카테고리 포함하여 출력"""
    item_count = defaultdict(int)
    
    # 타겟 상품의 이름과 카테고리 찾기
    target_item_name = item_profiles.get(target_item_id, {}).get('name', 'Unknown')
    target_item_categories = item_profiles.get(target_item_id, {}).get('categories', [])
    target_category_path = get_category_path_string(target_item_categories)
    
    # 타겟 상품 정보 출력
    print(f"\n타겟 상품: {target_item_name}, 카테고리: {target_category_path}")
    
    # 각 주문 파일에서 데이터를 읽고 함께 주문된 상품을 찾음
    for orders_path in orders_paths:
        df = pd.read_csv(orders_path)
        
        # 주문 번호별로 그룹화하여 함께 주문된 상품 확인
        grouped_orders = df.groupby('주문번호')['상품ID'].apply(list)
        
        for order in grouped_orders:
            if target_item_id in order:
                for item_id in order:
                    if item_id != target_item_id:
                        item_count[item_id] += 1
    
    # 주문 빈도수를 기준으로 내림차순 정렬
    sorted_items = sorted(item_count.items(), key=lambda x: x[1], reverse=True)
    
    # 상위 top_n개의 상품과 그 카테고리를 출력
    results = []
    for item_id, count in sorted_items:
        if item_id in item_profiles:
            item_name = item_profiles[item_id].get('name', 'Unknown')
            
            # 숫자 필터링 적용
            if item_name.endswith(str(number_filter)):
                item_categories = item_profiles[item_id].get('categories', [])
                category_path = get_category_path_string(item_categories)
                
                results.append({
                    'item_id': item_id,
                    'name': item_name,
                    'categories': category_path,
                    'count': count
                })
        
        # top_n 개수만큼 결과 제한
        if len(results) >= top_n:
            break
    
    # 출력
    print(f"\n상품 ID {target_item_id}와 함께 많이 주문된 상품 (이름이 {number_filter}로 끝나는 상품):")
    for result in results:
        print(f"상품 ID: {result['item_id']}, 이름: {result['name']}, 카테고리: {result['categories']}, 주문 빈도: {result['count']}")
    
    return results

# JSON 파일에서 item_profiles를 불러오는 함수
def load_item_profiles(json_path):
    """JSON 파일에서 item_profiles를 불러옵니다."""
    with open(json_path, 'r', encoding='utf-8') as f:
        item_profiles = json.load(f)
    return item_profiles

# 실행 예시
if __name__ == "__main__":
    # 상품 프로필 로드
    item_profiles_path = 'item_profiles_with_embeddings.json'
    item_profiles = load_item_profiles(item_profiles_path)
    
    # 주문 파일 리스트
    orders_paths = ['order1_df.csv', 'order2_df.csv', 'order3_df.csv', 'order4_df.csv', 'order5_df.csv']
    
    # 특정 상품 ID에 대해 많이 구매된 상품을 찾기
    target_item_id = input("Enter the item ID to find frequently bought together items: ")
    
    # 숫자 필터 입력
    number_filter = input("Enter a number (1-5) to filter items by name ending: ")
    if not number_filter.isdigit() or int(number_filter) not in range(1, 6):
        print("Invalid number filter. Please enter a number between 1 and 5.")
    else:
        number_filter = int(number_filter)
        find_frequently_bought_together(orders_paths, item_profiles, target_item_id, number_filter)

